**Author**

Shamim, 
Remote Backend Engineer at Short Circuit Science<br>
London, UK <br>
[GitHub](https://github.com/anamulislamshamim) [LinkedIn](https://www.linkedin.com/in/anamul-islam-shamim/)

**What is a Generator**

A generator is a special type of iterator in Python that lets you produce values on the fly, instead of storing them all in memory at once.

`Generator` define using the `yield` keyword (instead of return). Every time the function yield, execution is paused, and resume from the same point when called again.

Generator are memory-efficient and lazy (Compute value only when needed).

**Why use Generator?**

* Memory Efficiency: Handle large datasets without loading everything into RAM. Example: Reading a huge log file line by line.
* Performance: Lazily evaluate results -> faster startup, avoid unnecessary computation. 
* Streams & Pipelines: Process data continuously (e.g. handling live data, infinite sequences).

**Example: count up to n**

In [1]:
def count_to_n(n):
    count = 1
    while count <= n:
        yield count 
        count += 1

In [3]:
for num in count_to_n(10):
    print(num, end=" ")

1 2 3 4 5 6 7 8 9 10 

**How Generator Works**

The `yield` statement is fundamental to creating generators in Python. To understand its low-level implementation, we need to see how it changes the behavior of a function compared to a normal function using return.

**General function with return**

In [12]:
def square(n):
    numbers = []
    i = 1
    while i <= n:
        numbers.append(i * i)
        i += 1
    
    return numbers

In [21]:
for num in square(10):
    print(num, end=" ")

1 4 9 16 25 36 49 64 81 100 

**Implement square with iterator**

In [15]:
def square_iterator(n):
    i = 1
    while i <= n:
        yield i * i 
        i += 1

In [16]:
for num in square_iterator(10):
    print(num, end=" ")

1 4 9 16 25 36 49 64 81 100 

It seems the results are same right? Then what is the difference? 

1. In the first solution where we implement a function with reutrn will calculate all the squares from 1 to n first.
2. In this second solution I mean with yield will not calculate the square of 1 to n all together. Rather, it will calculate square for each of number during each iteration of the loop. That means it will act like API (Application Programming Interface)!

**Implement Generator without Yield**

In [19]:
class GeneratorSquare:
    def __init__(self, n):
        self.n = n
        self.i = 1
    
    # __iter__ method: Returns the iterator object (itself). 
    # This is required for the loop to work.
    def __iter__(self):
        return self
    
    def __next__(self):
        # calculate ith square value
        if self.i < self.n:
            result = self.i * self.i 
            self.i += 1
            return result
        else:
            # Raising StopIteration tells the loop to end
            raise StopIteration 

In [20]:
for i in GeneratorSquare(10):
    print(i, end=" ")

1 4 9 16 25 36 49 64 81 

**Read large file with generator**

In [22]:
def read_large_file(file_path):
    with open(file_path, "r") as file:
        for line in file:
            yield line.strip()

In [26]:
count = 1
iterator = read_large_file("large_log.txt")
for line in iterator:
    if count == 10:
        break
    print(line)
    count += 1

2023-11-26 10:00:00.123 | INFO | api-gateway | Request ID: 9f8a3d | Incoming GET request for /api/v1/users/456 from 192.168.1.101
2023-11-26 10:00:00.201 | DEBUG | auth-service | Token 'eyJhb...' validated successfully for user_id: 456
2023-11-26 10:00:00.355 | INFO | user-service | Fetching user profile for ID 456
2023-11-26 10:00:00.412 | DEBUG | db-connector | SQL: SELECT name, email FROM users WHERE id = 456
2023-11-26 10:00:00.509 | INFO | api-gateway | Request ID: 9f8a3d | Response 200 OK sent in 386ms
2023-11-26 10:00:00.678 | INFO | email-worker | Worker starting batch job for newsletters
2023-11-26 10:00:00.890 | INFO | api-gateway | Request ID: a1b2c3 | Incoming POST request for /api/v1/products/create
2023-11-26 10:00:01.050 | DEBUG | product-service | Attempting to validate product data: {"name": "Widget A", "price": 19.99}
2023-11-26 10:00:01.120 | WARN | db-connector | Slow query detected (400ms): INSERT INTO products...


In [27]:
for line in iterator:
    if count == 15:
        break 
    print(line)
    count += 1

2023-11-26 10:00:01.300 | INFO | api-gateway | Request ID: a1b2c3 | Response 201 Created sent in 410ms
2023-11-26 10:00:01.450 | ERROR | user-service | Failed to connect to external license API: Connection refused
2023-11-26 10:00:01.551 | INFO | inventory-sync | Sync job started. Source: SAP, Destination: Internal DB
2023-11-26 10:00:01.690 | DEBUG | inventory-sync | Processing item 55 of 1024. SKU: XQ-332
2023-11-26 10:00:01.880 | INFO | api-gateway | Request ID: d4e5f6 | Incoming GET request for /api/v1/metrics


**Generate Infinite Sequence with Generator**

In [28]:
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [30]:
fib = fibonacci()
for _ in range(6):
    print(next(fib), end=" ")

0 1 1 2 3 5 

In [31]:
for _ in range(3):
    print(next(fib), end=" ")

8 13 21 

Observation: Do you see that generator did not pre-compute ith sequence. It only produce sequence value when we need right?

**Interview Question**

What is a `generator` in Python and why would you use it?

Ans: “A generator is a special kind of iterator defined using yield. Unlike normal functions that return a single value and exit, a generator produces values one at a time, pausing between each. This makes them very memory-efficient, since they don’t store the whole dataset in memory. For example, I might use a generator to read a large log file line by line or to generate an infinite sequence like Fibonacci numbers.”

**List vs Generator (Memory Usage)**

In [34]:
import sys

# list Comprehension
numbers_list = [x for x in range(10_000_000)]
# Generator expression
numbers_gen = (x for x in range(10_000_000))

print("List Size:", sys.getsizeof(numbers_list)//1_000_000, "MB")
print("Generator size:", sys.getsizeof(numbers_gen), "Bytes")

List Size: 89 MB
Generator size: 192 Bytes


**Result Evaluation**
1. 👉 A list stores all 1 million numbers in memory.
2. 👉 A generator only stores the logic + current state (constant small memory).

**When to Use Which?**
1. List → When you need to access elements multiple times random access, or total length.
2. Generator → When working with very large or infinite sequences, streams, or when memory is a concern.